In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
#!pip uninstall keras-nightly

In [3]:
#!pip install tensorflow
#!pip install keras==2.12.*

In [4]:
from google.colab import drive
drive.mount('/content/drive') # Outputs will be saved in your google drive

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

import statsmodels
import statsmodels.tsa.api as sm
from statsmodels.tsa.api import VAR

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import matplotlib.dates as mdates

from datetime import datetime
from sklearn import preprocessing

In [6]:
import tensorflow as tf
from tensorflow import keras as keras

from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.layers import Layer
from keras import activations
from keras import initializers
from keras.layers import Dense, LSTM, SimpleRNN, Flatten

from keras.models import Sequential
#from tensorflow.keras.optimizers import SGD
#from keras.utils.np_utils import to_categorical

import tensorflow as tf

In [7]:
# not sure which random seeds are relevant below (just in case included them all)
#keras.utils.set_random_seed(42)
#np.random.seed(42)
#tf.random.set_seed(42)

tf.get_logger().setLevel('INFO')

In [8]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


The loss function is used to optimize your model. This is the function that will get minimized by the optimizer.

A metric is used to judge the performance of your model. This is only for you to look at and has nothing to do with the optimization process.

In [9]:
input_path_original = '/content/drive/MyDrive/Colab Notebooks/Gradu/Data/Original inflation/' # this contains original inflation values that are needed for reversin the stationarity transformation later
input_path = '/content/drive/MyDrive/Colab Notebooks/Gradu/Data/Features added/'

In [10]:
countries = ['Finland', 'United States', 'Germany', 'France', 'Spain', 'Italy', 'Netherlands', 'Sweden', 'Belgium', 'Denmark', 'Austria', 'Poland']
#countries = ['Finland']
transformations = ['FD', 'FD', 'none', 'FD', 'FD', 'FD', 'FD', 'none', 'none', 'FD', 'none', 'FD']
#transformations = ['FD']

In [11]:
macro_variables = ['inflation', 'unemployment', 'imports', 'exports', 'bond_yield', 'exchange', 'ppi', 'bci', 'cci', 'construction', 'manufacturing', 'share_prices', 'gdp', 'house_prices', 'investment', 'domestic_demand']
common_variables = ['oil', 'silver', 'eurusd', 'eurcfh', 'spx', 'world']

all_variables = macro_variables + common_variables

In [12]:
def get_data(variable, path_in = input_path):
    input_file = path_in + variable +'.csv'
    data = pd.read_csv(input_file, header=0, index_col=0)
    data.index = pd.to_datetime(data.index)
    data = data.astype(float)
    data = data.sort_index()
    return data

In [13]:
# drop constant columns

def drop_constant_columns(data):
    non_constant_columns = data.columns[data.nunique() > 1]  # Find non-constant columns
    return data[non_constant_columns].copy()  # Return new DataFrame with non-constant columns

In [14]:
# create the matrix of variables

# initialise dataframe with hicp_Finland data after which more columns are added into it
init_index = get_data("inflation_Finland", input_path)
df = pd.DataFrame(index = init_index.index)

for variable in macro_variables:
    for country in countries:
        data = get_data(variable + "_" + country)
        new_column_name = variable + "_" + country
        data = data.rename({'Data': new_column_name}, axis=1)
        df[new_column_name] = data

for variable in common_variables:
    data = get_data(variable + "_" + 'Finland')
    new_column_name = variable + "_" + 'common'
    data = data.rename({'Data': new_column_name}, axis=1)
    df[new_column_name] = data
#df = df.replace(np.inf, np.nan).replace(-np.inf, np.nan).dropna()

df = drop_constant_columns(df)
print(df.shape)
print(df)

<ipython-input-14-b86d5496bf3f>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = data
<ipython-input-14-b86d5496bf3f>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = data
<ipython-input-14-b86d5496bf3f>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  d

(276, 196)
            inflation_Finland  inflation_United States  inflation_Germany  \
Date                                                                        
2000-01-01           0.025006                 0.013511            1.54946   
2000-02-01           0.512560                 0.482990            1.54762   
2000-03-01           0.364720                 0.535700            1.54762   
2000-04-01          -0.503340                -0.688990            1.06635   
2000-05-01           0.053520                 0.120340            0.94787   
...                       ...                      ...                ...   
2022-08-01          -0.169370                -0.262130            6.95652   
2022-09-01           0.503220                -0.061020            8.57418   
2022-10-01           0.191470                -0.456240            8.82071   
2022-11-01           0.827470                -0.635110            8.80383   
2022-12-01           0.006800                -0.655920           

<ipython-input-14-b86d5496bf3f>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = data


In [15]:
def split_sequences(sequences, no_steps_in, no_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        #find end of pattern
        end_ix = i + no_steps_in
        out_end_ix = end_ix + no_steps_out - 1

        #check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break

        #gather input and output
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix - 1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)

    return np.array(X), np.array(y)

# Model implementation

In [19]:
def create_LSTM(no_features, no_steps_in, no_steps_out):
    units = 128 # 100?
    model = Sequential()
    model.add(LSTM(units, activation = 'tanh', return_sequences = True, input_shape = (no_steps_in, no_features)))
    model.add(LSTM(units, activation = 'tanh'))
    model.add(Dense(no_steps_out))
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.001),loss = 'mse')
    return model

# Forecasting for 2022

In [20]:
steps_in = 12 # use previous 12 observations to predict the next horizon
features = df.shape[1]

#countries = ['Finland']
countries = ['Finland', 'United States', 'Germany', 'France', 'Spain', 'Italy', 'Netherlands', 'Sweden', 'Belgium', 'Denmark', 'Austria', 'Poland']


for (country, transformation) in zip(countries,transformations):
    warnings.filterwarnings('ignore')

    print("country: " + str(country))
    print()

    # data (original) for reversing stationarity and calculating the RMSE
    split_point = len(df)-12
    original_inflation = "original_inflation_" + str(country)
    data_original = get_data(original_inflation, input_path_original)
    test_original = data_original[split_point:] # original inflation for 2022

    # train data
    train = np.array(df.loc['2000-01-01':'2021-12-01'])
    train_out = np.array(df.loc['2000-01-01':'2021-12-01']['inflation_' + country])

    # normalise features
    scaler = StandardScaler()
    scaler.fit(train)
    normalized_train = scaler.transform(train)

    #normalized_train_out = target_scaler.transform(train_out)
    #normalized_train_out = normalized_train_out.reshape(len(normalized_train_out), 1)

    # normalise target (needed especially for time series that aren't differenced because otherwise the scales are too different?)
    target_scaler = StandardScaler()
    train_out = train_out.reshape(len(train_out), 1)
    target_scaler.fit(train_out)
    normalized_train_out = target_scaler.transform(train_out)

    train_set = np.hstack((normalized_train, normalized_train_out))

    # out-of-sample-inputs
    test = np.array(df.loc['2021-01-01':'2021-12-01'])
    normalized_test = scaler.transform(test)
    test_set = normalized_test.reshape((1, steps_in, features))

    horizons = [1,2,3,6,9,12]

    for horizon in horizons:
        keras.utils.set_random_seed(42) # set random seed for reproducability

        warnings.filterwarnings('ignore')
        print("horizon: " + str(horizon))
        horizon_test = test_original[:horizon]
        forecast_df = pd.DataFrame(index=horizon_test.index)
        forecast_df['Actual'] = horizon_test['Data']

        X,y = split_sequences(train_set, steps_in, horizon)

        es = EarlyStopping(monitor = 'val_loss', mode = 'min', min_delta = 0.01, patience = 10,verbose = 0)
        model = create_LSTM(features, steps_in, horizon) # the model is trained specifically to forecast the horizon length
        print(model.summary())
        fit = model.fit(X, y, verbose=0, epochs=500, validation_split=0.1, callbacks = [es])

        forecast_normalized = model.predict(test_set, verbose = 0)
        forecast_denormalized = target_scaler.inverse_transform(forecast_normalized)
        forecast = forecast_denormalized.flatten()

        # we need to reverse the stationarity
        if transformation == "FD":
            print("Reversing the stationarity...")
            last_observed_value = data_original[:split_point]
            last_observed_value = last_observed_value.values
            last_observed_value = last_observed_value[-1] # get the last observed value, i.e. 2021-12

            forecasted_original = []
            for pred in forecast:
                forecasted_value = last_observed_value + pred
                forecasted_original.append(forecasted_value)
                last_observed_value = forecasted_value
            forecast = forecasted_original
            forecast = np.concatenate(forecast)
            forecast_df['Forecast'] = forecast
        else:
            forecast_df['Forecast'] = forecast

        print
        print("forecast:")
        print(forecast_df)
        print()

        rmse = np.sqrt(mean_squared_error(forecast_df['Actual'], forecast_df['Forecast']))
        print('RMSE: ', rmse)
        print()
        print("######")

    print()
    print("##################")

country: Finland

horizon: 1
Model: "sequential_72"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_144 (LSTM)             (None, 12, 128)           166400    
                                                                 
 lstm_145 (LSTM)             (None, 128)               131584    
                                                                 
 dense_72 (Dense)            (None, 1)                 129       
                                                                 
Total params: 298113 (1.14 MB)
Trainable params: 298113 (1.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Reversing the stationarity...
forecast:
             Actual  Forecast
Date                         
2022-01-01  4.37452  3.604728

RMSE:  0.7697918595743185

######
horizon: 2
Model: "sequential_73"
________________________________________________

### Forecasting for 2022 full year (old code for reference)

In [18]:
# 80/10/10 split is usually recommended
# here could do that 2 years for validation and 1 year for out-of-sample testing

train = np.array(df.loc['2000-01-01':'2019-12-01'])
val = np.array(df.loc['2020-01-01':'2021-12-01'])

out = np.array(df.loc['2000-01-01':'2019-12-01']['hicp_Finland_LD'])
val_out = np.array(df.loc['2020-01-01':'2021-12-01']['hicp_Finland_LD'])

out = out.reshape(len(out), 1)
val_out = val_out.reshape(len(val_out), 1)

# out-of-sample inputs
test_set = np.array(df.loc['2021-01-01':'2021-12-01'])

KeyError: ignored

In [ ]:
# scale
#scaler = preprocessing.StandardScaler()
#scaler.fit(train)

#train_set = scaler.transform(train)
#val_set = scaler.transform(val)
#test_set = scaler.transform(test_set)

train_set = np.hstack((train, out))
val_set = np.hstack((val, val_out))

In [ ]:
# timesteps
no_steps_in, no_steps_out = 12, 12 # use 12 data points to predict the next 12 data points

train_X, train_y = split_sequences(train_set, no_steps_in, no_steps_out)
print(train_X.shape, train_y.shape)
val_X, val_y = split_sequences(val_set, no_steps_in, no_steps_out)
print(val_X.shape, val_y.shape)
test_set = test_set.reshape((1, no_steps_in, no_variables))
print(test_set.shape)

In [ ]:
model = create_LSTM(no_variables, no_steps_in, no_steps_out)

es = EarlyStopping(monitor = 'val_loss', mode = 'min', min_delta = 0.01, patience = 30, verbose = 0)

fit = model.fit(train_X, train_y, validation_data = (val_X, val_y), epochs = 500, verbose = 0, callbacks = [es])

In [ ]:
print(model.summary())

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(fit.history['loss'], color = 'blue')
ax.plot(fit.history['val_loss'], label = 'red', color = 'DarkRed')
ax.legend(["training", "validation"])
fig.set_size_inches(12,3)

In [ ]:
actual = df.loc['2022-01-01':'2022-12-01']['hicp_Finland_LD']

In [ ]:
pred = model.predict(test_set, verbose = 0)
print(pred.shape)
pred = pred.reshape(pred.shape[1])
pred = (pd.DataFrame(pred, index=actual.index))
print(pred.shape)
print(pred)

In [ ]:
#pred = np.transpose([pred] * no_variables) # need to inverse transform and hence same shape is required as when it was transformed
#pred = scaler.inverse_transform(pred)
#pred = pred[:,0]
#pred = (pd.DataFrame(pred, index=actual.index))

In [ ]:
rmse = np.sqrt(mean_squared_error(actual, pred))
print('Test RMSE: %.6f' % rmse)

fig, ax = plt.subplots(1,1)
ax.plot(actual, color='blue')
ax.plot(pred, color='red')
ax.legend(['actual', 'predicted'])
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.set_xlim(actual.index[0], actual.index[len(actual)-1])
#ax.set_title("HICP, Finland")
#ax.set_ylabel("Index, 2015")
#ax.set_xlim('2022-01-01', '2022-12-01')
fig.set_size_inches(12,3)

### Forecasting for 2019

In [ ]:
train = np.array(df.loc['2000-01-01':'2016-12-01'])
val = np.array(df.loc['2017-01-01':'2018-12-01'])

out = np.array(df.loc['2000-01-01':'2016-12-01']['hicp_Finland_LD'])
val_out = np.array(df.loc['2017-01-01':'2018-12-01']['hicp_Finland_LD'])

out = out.reshape(len(out), 1)
val_out = val_out.reshape(len(val_out), 1)

# out-of-sample inputs
test_set = np.array(df.loc['2018-01-01':'2018-12-01'])

In [ ]:
# scale
#scaler = preprocessing.StandardScaler()
#scaler.fit(train)

#train_set = scaler.transform(train)
#val_set = scaler.transform(val)
#test_set = scaler.transform(test_set)

train_set = np.hstack((train, out))
val_set = np.hstack((val, val_out))

In [ ]:
# scale
#scaler = preprocessing.StandardScaler()
#scaler.fit(train)

#train_set = scaler.transform(train)
#val_set = scaler.transform(val)
#test_set = scaler.transform(test_set)

train_set = np.hstack((train, out))
val_set = np.hstack((val, val_out))

In [ ]:
# timesteps
no_steps_in, no_steps_out = 12, 12 # use 12 data points to predict the next 12 data points

train_X, train_y = split_sequences(train_set, no_steps_in, no_steps_out)
print(train_X.shape, train_y.shape)
val_X, val_y = split_sequences(val_set, no_steps_in, no_steps_out)
print(val_X.shape, val_y.shape)
test_set = test_set.reshape((1, no_steps_in, no_variables))
print(test_set.shape)

In [ ]:
model = create_LSTM(no_variables, no_steps_in, no_steps_out)

es = EarlyStopping(monitor = 'val_loss', mode = 'min', min_delta = 0.01, patience = 30, verbose = 0)

fit = model.fit(train_X, train_y, validation_data = (val_X, val_y), epochs = 500, verbose = 0, callbacks = [es])

In [ ]:
print(model.summary())

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(fit.history['loss'], color = 'blue')
ax.plot(fit.history['val_loss'], label = 'red', color = 'DarkRed')
ax.legend(["training", "validation"])
fig.set_size_inches(12,3)

In [ ]:
actual = df.loc['2019-01-01':'2019-12-01']['hicp_Finland_LD']

In [ ]:
pred = model.predict(test_set, verbose = 0)
print(pred.shape)
pred = pred.reshape(pred.shape[1])
pred = (pd.DataFrame(pred, index=actual.index))
print(pred.shape)
print(pred)

In [ ]:
rmse = np.sqrt(mean_squared_error(actual, pred))
print('Test RMSE: %.6f' % rmse)

fig, ax = plt.subplots(1,1)
ax.plot(actual, color='blue')
ax.plot(pred, color='red')
ax.legend(['actual', 'predicted'])
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.set_xlim(actual.index[0], actual.index[len(actual)-1])
fig.set_size_inches(12,3)